In [1]:
import seaborn as sns
import networkx
import folium
import pandas as pd
import osmnx as ox
import ipywidgets as widgets
from folium import plugins
import ipyleaflet as ipl
from ipywidgets import Layout
from time import sleep
ox.config(use_cache=True, log_console=True)

2022-03-10 16:46:20 Configured OSMnx 1.1.2
2022-03-10 16:46:20 HTTP response caching is on


In [2]:
!jupyter serverextension enable voila
!jupyter server extension enable voila

Enabling: voila
- Writing config: /Users/witoldtenhove/.jupyter
    - Validating...
      voila 0.3.1 OK
Enabling: voila
- Writing config: /Users/witoldtenhove/opt/anaconda3/envs/osmnx/etc/jupyter
    - Validating voila...
      voila 0.3.1 OK
    - Extension successfully enabled.


## Load data

In [2]:
url = 'routes_data.csv'
routes_df = pd.read_csv(url)
routes_df.head()

,route_id,arrival,activity_id,shift_id,longitude,latitude
0,0,1313.393,77.0,6.0,6.164641,52.269232
1,0,1313.393,68.0,5.0,6.164916,52.269976
2,0,1381.537,1.0,0.0,6.164124,52.269695
3,0,1440.034,50.0,3.0,6.172841,52.265654
4,0,2281.557,79.0,6.0,6.172033,52.266394


## Create dictionary

In [40]:

deventer_graph = ox.load_graphml("deventer_graph.graphml")
gdf_nodes = ox.graph_to_gdfs(deventer_graph)[0]
route_ids = pd.unique(routes_df.route_id.values)
routes_dict = {}
# Add base data to dictionary. Main key is route_id.
for id in route_ids:
    route_items = routes_df[routes_df['route_id'] == id]
    route_dict = {}
    for location in route_items.itertuples():
        index, route_id, arrival, activity_id, shift_id, longitude, latitude = location
        route_dict[index] = {'route_id': route_id,
                                    'arrival': arrival,
                                    'activity_id': activity_id,
                                    'shift_id': shift_id,
                                    'location': [latitude, longitude]
                                    }
    routes_dict[id] = route_dict

# For each location in route calculate the path from the previous location. Start at a base.
base = [52.26, 6.15]
for r in routes_dict.keys():
    cnt = 0
    for i in routes_dict[r].keys():
        if cnt == 0:
            routes_dict[r][i]['start'] = base
        else:
            routes_dict[r][i]['start'] = routes_dict[r][i-1]['location']
        y1, x1 = routes_dict[r][i]['start']
        y2, x2 = routes_dict[r][i]['location']
        nodes = ox.nearest_nodes(G=deventer_graph, X = [x1, x2], Y = [y1, y2])
        path_nodes = networkx.shortest_path(deventer_graph, nodes[0], nodes[1])
        path_coord = gdf_nodes.loc[path_nodes][['x', 'y']]
        path = []
        for point in path_coord.values:
            path.append([point[1], point[0]])
        routes_dict[r][i]['path'] = path
        cnt += 1
routes_dict

2022-03-10 17:24:57 Converting node, edge, and graph-level attribute data types
2022-03-10 17:24:57 Loaded graph with 20870 nodes and 39886 edges from "deventer_graph.graphml"
2022-03-10 17:24:58 Created nodes GeoDataFrame from graph
2022-03-10 17:25:01 Created edges GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 17:25:02 Created nodes GeoDataFrame from graph
2022-03-10 1

{0: {0: {'route_id': 0,
   'arrival': 1313.3929999999998,
   'activity_id': 77.0,
   'shift_id': 6.0,
   'location': [52.26923206571011, 6.164640954725753],
   'start': [52.26, 6.15],
   'path': [[52.2600292, 6.1498598],
    [52.2598294, 6.1491946],
    [52.2602488, 6.1488088],
    [52.2606732, 6.1500892],
    [52.2602743, 6.1504512],
    [52.2602324, 6.1504951],
    [52.2605749, 6.1511428],
    [52.26092, 6.15174],
    [52.2609668, 6.1518081],
    [52.2610119, 6.1519246],
    [52.2611166, 6.1520763],
    [52.2611407, 6.1521046],
    [52.2611807, 6.1521481],
    [52.2612071, 6.1521691],
    [52.2612217, 6.1521726],
    [52.2612349, 6.1521703],
    [52.2612624, 6.1521561],
    [52.2612912, 6.152142],
    [52.2613198, 6.1521382],
    [52.2613683, 6.1521618],
    [52.2616206, 6.1522948],
    [52.2620292, 6.152515],
    [52.2623602, 6.1526873],
    [52.2624708, 6.152745],
    [52.2626401, 6.1528394],
    [52.2634167, 6.1532788],
    [52.26366, 6.15341],
    [52.2644, 6.1538559],
    [52.26

## Create base map

In [6]:
base_map = folium.Map(location=[routes_df.latitude.mean(
), routes_df.longitude.mean()], zoom_start=15, control_scale=True)
base_map

## Plot all client locations by route (colors)

In [29]:
# Using dictionary

n_colors = len(pd.unique(routes_df.route_id.values))
colors = sns.color_palette("Set2", n_colors).as_hex()

m = base_map
for routes in routes_dict.values():
        for point in routes.values():
                folium.Circle(
                        point["location"],
                        radius=30,
                        color="DimGray",
                        fill_color=colors[int(point["route_id"])],
                        fill=True,
                        fill_opacity=0.85,
                        popup=point["route_id"]
                ).add_to(m)
m

In [7]:
n_colors = len(pd.unique(routes_df.route_id.values))
colors = sns.color_palette("Set2", n_colors).as_hex()

m = base_map
for index, location_info in routes_df.iterrows():
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=30,
        color="DimGray",
        fill_color=colors[int(location_info["route_id"])],
        fill=True,
        fill_opacity=0.85,
        popup=location_info["route_id"]
    ).add_to(m)
m

## Trace routes on map

In [7]:
deventer_graph = ox.load_graphml("deventer_graph.graphml")

## Function to trace a single route (by route_id) on map

def calc_route(route_id: int, path_coordinates, graph):
    route_id = route_id
    # Get arrays of longitudes and latitudes for path for a given rout_id
    lon_array = path_coordinates[path_coordinates.route_id ==
                                 route_id].longitude
    lat_array = path_coordinates[path_coordinates.route_id ==
                                 route_id].latitude
    gdf_nodes = ox.graph_to_gdfs(graph)[0]

    # Calculates nearest nodes for all path coordinates from arrays
    node_list = ox.distance.nearest_nodes(
        G=deventer_graph,
        X=lon_array,
        Y=lat_array
    )

    # for each pair of subsequent nodes calculate shortest path and append array of path nodes to route array
    route = [[], []]
    for i in range(len(node_list) - 1):
        start = node_list[i]
        end = node_list[i + 1]
        path_nodes = networkx.shortest_path(graph, start, end)
        route[0].append(path_nodes)
        route[1].append(gdf_nodes.loc[path_nodes][['x','y']])
        i += 1

    # return route as list
    return(route)

2022-02-28 11:52:35 Converting node, edge, and graph-level attribute data types
2022-02-28 11:52:35 Loaded graph with 20870 nodes and 39886 edges from "deventer_graph.graphml"


## Plot route on map

In [8]:
route_id = 5

route_list = calc_route(route_id, routes_df, deventer_graph)[0]
route_df_flr = routes_df[routes_df['route_id'] == route_id]
route_map = folium.Map(location=[route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()], control_scale=True)
for index, location_info in route_df_flr.iterrows():
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=20,
        color="DimGray",
        fill_color=colors[int(location_info["route_id"])],
        fill=True,
        fill_opacity=0.85,
        popup=round(location_info["arrival"])
    ).add_to(route_map)

for i, stage in enumerate(route_list):
    route_map = ox.folium.plot_route_folium(
        G=deventer_graph, route=stage, route_map=route_map, color='red', weight=2)

sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
route_map.fit_bounds([sw, ne])
route_map



2022-02-28 11:52:36 Created nodes GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created nodes GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:52:39 Created edges GeoDataFrame from graph
2022-02-28 11:

In [9]:

def trace_route(route_id):
    route_list = calc_route(route_id, routes_df, deventer_graph)[0]
    route_df_flr = routes_df[routes_df['route_id'] == route_id]
    route_map_i = folium.Map(location=[route_df_flr.latitude.mean(
    ), route_df_flr.longitude.mean()], control_scale=True)
    for index, location_info in route_df_flr.iterrows():
        folium.Circle(
            [location_info["latitude"],
             location_info["longitude"]],
            radius=20,
            color="DimGray",
            fill_color=colors[int(location_info["route_id"])],
            fill=True,
            fill_opacity=0.85,
            popup=round(location_info["arrival"])
        ).add_to(route_map_i)

    for i, stage in enumerate(route_list):
        route_map_i = ox.folium.plot_route_folium(
            G=deventer_graph, route=stage, route_map=route_map_i, color='red', weight=2)

    sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
    ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
    route_map_i.fit_bounds([sw, ne])
    display(route_map_i)

In [10]:
route_ids = pd.unique(routes_df.route_id.values)

routeWidget = widgets.RadioButtons(
    options=route_ids,
    description='RouteID:',
    disabled=False
)
    
widgets.interactive(trace_route, route_id=routeWidget)



interactive(children=(RadioButtons(description='RouteID:', options=(0, 1, 2, 3, 4, 5, 6), value=0), Output()),…

## Ant path

In [11]:

# create map

route_id = 5

route_list = calc_route(route_id, routes_df, deventer_graph)[1]
point_list = []
for stage in route_list:
    for point in stage.values:
        point_list.append([point[1], point[0]])
      
route_df_flr = routes_df[routes_df['route_id'] == route_id]
print(route_df_flr)

ant_route_map = folium.Map(location=[route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()], control_scale=True)

for index, location_info in route_df_flr.iterrows():
    fill_color = colors[int(location_info["route_id"])]
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85,
        popup=round(location_info["arrival"])
    ).add_to(ant_route_map)

# Ploting ant-route
ap = plugins.AntPath(point_list, delay='1500', dash_array=['5', '50'], color='Yellow', pulse_color='Red')
ap.add_to(ant_route_map)

start = folium.Marker(
    point_list[0], tooltip=folium.Tooltip('Start', permanent=True)
)
start.add_to(ant_route_map)

finish = folium.Marker(
    point_list[-1], tooltip=folium.Tooltip('Finish', permanent=True)
)
finish.add_to(ant_route_map)

sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
ant_route_map.fit_bounds([sw, ne])
display(ant_route_map)

2022-02-28 11:52:45 Created nodes GeoDataFrame from graph
2022-02-28 11:52:48 Created edges GeoDataFrame from graph
2022-02-28 11:52:48 Created nodes GeoDataFrame from graph
    route_id   arrival  activity_id  shift_id  longitude   latitude
62         5   420.000         58.0       4.0   6.160594  52.267430
63         5   420.000         36.0       2.0   6.161268  52.267151
64         5  1183.406         20.0       1.0   6.165260  52.267300
65         5  2162.731         52.0       3.0   6.166464  52.269238
66         5  2779.272         71.0       5.0   6.166425  52.269844
67         5  3311.956         60.0       4.0   6.169059  52.271503
68         5  3696.765         24.0       1.0   6.168793  52.270316
69         5  3912.071         25.0       1.0   6.167334  52.268616
70         5  4289.686          5.0       0.0   6.166916  52.267690
71         5  4537.609          6.0       0.0   6.171699  52.269982
72         5  5046.272         63.0       4.0   6.170195  52.268510
73        

# Ipyleaflet

In [49]:
# Using dictionary

route_id = 4

route_dict_flr = routes_dict[route_id]

# Calculate map center
latitudes = pd.Series(dtype="float64")
longitudes = pd.Series(dtype="float64")
for point in route_dict_flr.values():
    lat, lon = pd.Series([point["location"][0]]), pd.Series([point["location"][1]])
    latitudes = pd.concat([latitudes, lat])
    longitudes = pd.concat([longitudes, lon])
center = [latitudes.mean(), longitudes.mean()]
m1 = ipl.Map(center=center)

# Buid route layer
trace = []
for point in route_dict_flr.values():
    for path in point["path"]:
        trace.append(path)
lines = ipl.Polyline(
    locations=trace,
    color="tomato",
    fill=False
)
m1.add_layer(lines)

# Build client locations layer
base = list(route_dict_flr.values())[0]
print(base["start"])
start = ipl.Circle(
    location=base["start"],
    name=str(0),
    radius=20,
    color="Red",
    fill_color="Red",
    fill=True,
    fill_opacity=0.85
)
m1.add_layer(start)

for client in route_dict_flr.values():
    fill_color = colors[int(client["route_id"])]
    circle = ipl.Circle(
        location=client["location"],
        name=str(round(client["arrival"])),
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85
    )
    m1.add_layer(circle)

m1

[52.26, 6.15]


Map(center=[52.26849415073453, 6.166401483360303], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [171]:
route_id = 4

route_list = calc_route(route_id, routes_df, deventer_graph)[1]
point_list = []
for stage in route_list:
    for point in stage.values:
        point_list.append([point[1], point[0]])
print(point_list)
route_df_flr = routes_df[routes_df['route_id'] == route_id]

m1 = ipl.Map(center=([route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()]))

lines = ipl.Polyline(
    locations=point_list,
    color="tomato",
    fill=False
)

m1.add_layer(lines)

for index, location_info in route_df_flr.iterrows():
    fill_color = colors[int(location_info["route_id"])]
    circle = ipl.Circle(
        location=[location_info["latitude"],location_info["longitude"]],
        name=str(round(location_info["arrival"])),
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85
    )
    m1.add_layer(circle)

start_popup = ipl.Popup(
    location=point_list[0],
    child=widgets.HTML(value='Start'),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m1.add_layer(start_popup)

finish_popup = ipl.Popup(
    location = point_list[-1],
    child=widgets.HTML(value='Finish'),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m1.add_layer(finish_popup)
control = ipl.LayersControl(position='topright')
m1.add_control(control)
    
msw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
m1.fit_bounds([sw, ne])
display(m1)

m1.save('interactive_route_map.html', title='Route Map')


2022-03-10 16:34:34 Created nodes GeoDataFrame from graph
2022-03-10 16:34:37 Created edges GeoDataFrame from graph
2022-03-10 16:34:37 Created nodes GeoDataFrame from graph
[[52.2695902, 6.1765047], [52.2689551, 6.1764453], [52.2687178, 6.176425], [52.2686626, 6.1764269], [52.2684873, 6.1764242], [52.2684824, 6.1762049], [52.2684707, 6.1760168], [52.2684559, 6.1758205], [52.2684332, 6.1755801], [52.2683993, 6.1753956], [52.2683496, 6.1751902], [52.26832, 6.17506], [52.2682504, 6.1748312], [52.2682504, 6.1748312], [52.26832, 6.17506], [52.2683496, 6.1751902], [52.2683778, 6.1753957], [52.2683996, 6.1756129], [52.268397, 6.1760867], [52.2683947, 6.1762058], [52.2683913, 6.1764228], [52.2684873, 6.1764242], [52.2686626, 6.1764269], [52.2687178, 6.176425], [52.2689551, 6.1764453], [52.2695902, 6.1765047], [52.2702355, 6.1765497], [52.2702355, 6.1765497], [52.2695902, 6.1765047], [52.2689551, 6.1764453], [52.2687178, 6.176425], [52.2686626, 6.1764269], [52.2684873, 6.1764242], [52.2684824,

Map(center=[52.26849415073453, 6.166401483360303], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [30]:
route_id = 5

route_df_flr = routes_df[routes_df['route_id'] == route_id] # All clients on a particular route

route_list = calc_route(route_id, routes_df, deventer_graph)[1] # All path coordinates from client to client on a particular route
point_list = []
ix_list = []
for ix, stage in enumerate(route_list):
    for point in stage.values:
        point_list.append([point[1], point[0]])
        ix_list.append(ix)
path_list = [point_list, ix_list] 

m3 = ipl.Map(center=([route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()]), layout=Layout(height='900px'))

lines = ipl.Polyline(
    locations=path_list[0],
    color="tomato",
    fill=False
)

m3.add_layer(lines)
for index, location_info in route_df_flr.iterrows():
    fill_color = colors[int(location_info["route_id"])]
    circle = ipl.Circle(
        location=[location_info["latitude"],location_info["longitude"]],
        name=str(index),
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85
    )
    m3.add_layer(circle)
control = ipl.LayersControl(position='topright')
m3.add_control(control)

mark = ipl.Marker(location=path_list[0][0])
m3.add_layer(mark)
    
sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
m3.fit_bounds([sw, ne])
display(m3)

2022-02-28 17:06:07 Created nodes GeoDataFrame from graph
2022-02-28 17:06:11 Created edges GeoDataFrame from graph
2022-02-28 17:06:11 Created nodes GeoDataFrame from graph


Map(center=[52.2698088679728, 6.167843325392477], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [31]:
# Animation part
step = 0.2
state = None 
for ix, p in enumerate(path_list[0]):
    mark.location = p
    t = path_list[1][ix]
    if(t != state):
        state = t
        visited = ipl.Circle(
            location=[route_df_flr["latitude"].iloc[state], route_df_flr["longitude"].iloc[state]],
            radius=20,
            color="#00b521",
            fill_color="#7fff96",
            fill=True,
            fill_opacity=0.85
        )
        m3.add_layer(visited)
    sleep(step)
visited = ipl.Circle(
    location=[route_df_flr["latitude"].iloc[-1], route_df_flr["longitude"].iloc[-1]],
    radius=20,
    color="#00b521",
    fill_color="#7fff96",
    fill=True,
    fill_opacity=0.85
)
m3.add_layer(visited)
sleep(step+0.4)
m3.remove_layer(mark)